In [ ]:
# 常用api测试

In [1]:
import tensorflow as tf 
from tensorflow.python.ops import nn 

def sigmoid_cross_entropy_loss_v1(
        labels,
        logits,
        weights=1.0,
        label_smoothing=0,
        alpha=0.5,
        gamma=0,
        reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
        from_logits=True
):
    '''
    封装的功能：
    1) 最基本的二分类celoss, 支持from_logits 和 from_probs 两种形式
    2) label_smooth
    3) weighted celoss
    4) focal loss? 
    '''

    if labels is None:
        raise ValueError('labels must not be None.')
    if logits is None:
        raise ValueError('logits must not be None.')

    with tf.name_scope("sigmoid_cross_entropy_loss_v1") as scope:
        logits = tf.convert_to_tensor(logits)
        labels = tf.cast(labels, dtype=logits.dtype)
        weights = tf.cast(labels, dtype=logits.dtype)

        logits.get_shape().assert_is_compatible_with(labels.get_shape())

        smoothed_labels = labels * (1 - label_smoothing) + 0.5 * label_smoothing

        if from_logits:
            # return (b,1), 还需配合 reduce_xxx 变成一个scalar
            loss = nn.sigmoid_cross_entropy_with_logits(
                labels=smoothed_labels, logits=logits, name='cross_entropy'
            )
        else:
            epsilon_ = tf.constant(1e-7) # smooth term
            logits = tf.clip_by_value(logits, epsilon_, 1 - epsilon_) # clip to (0,1) since probs

            # 因为logits是probs,所以直接基于celoss公式计算loss
            bce = smoothed_labels * tf.log(logits + epsilon_)
            bce += (1 - smoothed_labels) * tf.log( 1 - logits + epsilon_)
            loss = bce 
        
        # focal loss
        if gamma > 0:
            alpha = tf.cast(alpha, dtype=loss.dtype)
            gamma = tf.cast(gamma, dtype=loss.dtype)

            probs = tf.sigmoid(logits, name='sigmoid') if from_logits else logits

            at = labels * alpha + (1 - labels) * (1 - alpha)
            pt = labels * probs + (1 - labels) * (1 - probs)

            loss = at * tf.pow((1 - pt), gamma) * loss 
        
        # weighted loss and reduce to scalar
        return tf.losses.compute_weighted_loss(
            loss, weights, scope, reduction=reduction
        )


 # ============================================================== #   
# 测试上述func
def test_v1():
    labels = None
    logits = []
    loss = sigmoid_cross_entropy_loss_v1(
        labels,
        logits,
    )
    with tf.Session() as sess:
        loss_res = sess.run(loss)
        print("test_celoss_v1 loss: ", loss_res)
# test_v1()

def test_v2():
    labels = []
    logits = []
    loss = sigmoid_cross_entropy_loss_v1(
        labels,
        logits,
    )
    with tf.Session() as sess:
        loss_res = sess.run(loss)
        print("test_celoss_v1 loss: ", loss_res)
test_v2()

test_celoss_v1 loss:  0.0
